In [1]:
import pathlib
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import papermill as pm

In [2]:
# Parameters
celltype = "SHSY5Y"

In [3]:
# Define inputs
feature_file = pathlib.Path(f"../data/{celltype}_sc_norm_fs.parquet")
feature_df = pq.read_table(feature_file).to_pandas()

In [ ]:
# fix a formatting error
# replace "2.5uM" with 2.5 in the 'Metadata_inducer1_concentration' column if it exists
if "Metadata_inducer1_concentration" in feature_df.columns:
    feature_df["Metadata_inducer1_concentration"] = feature_df[
        "Metadata_inducer1_concentration"
    ].replace("2.5uM", 2.5)

In [4]:
# define output file path
feature_df_out_path = pathlib.Path(f"../data/{celltype}_preprocessed_sc_norm.parquet")

In [5]:
print(feature_df.shape)
feature_df.head()

(597902, 1270)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-1.841853,-0.597438,-1.295016,-3.127056,-3.004511,-1.481117,1.851482,0.024721,0.307472,0.092086
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.841272,-0.681935,-0.330951,-3.725197,-0.827474,-0.461348,0.897731,-0.041156,1.443262,0.009843
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.500885,-0.229003,-1.254209,-0.691997,-1.374967,-1.337252,0.825970,-0.044386,-0.020445,0.000848
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.560136,-0.276664,-0.313186,-0.844206,-0.798213,-1.032992,0.546308,-0.058328,-0.009632,-0.005811
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.828550,-0.761657,-0.309806,-0.501700,-0.616252,-0.681687,1.034724,-0.035518,-0.038205,0.017690


In [6]:
# removing costes features as they behave with great variance across all data
feature_df = feature_df.drop(feature_df.filter(regex="Costes").columns, axis=1)
print(feature_df.shape)
feature_df.head()

(597902, 1270)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-1.841853,-0.597438,-1.295016,-3.127056,-3.004511,-1.481117,1.851482,0.024721,0.307472,0.092086
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.841272,-0.681935,-0.330951,-3.725197,-0.827474,-0.461348,0.897731,-0.041156,1.443262,0.009843
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.500885,-0.229003,-1.254209,-0.691997,-1.374967,-1.337252,0.825970,-0.044386,-0.020445,0.000848
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.560136,-0.276664,-0.313186,-0.844206,-0.798213,-1.032992,0.546308,-0.058328,-0.009632,-0.005811
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.828550,-0.761657,-0.309806,-0.501700,-0.616252,-0.681687,1.034724,-0.035518,-0.038205,0.017690


In [7]:
# replacing '/' in treatment dosage column to avoid errors in file interpolation including such strings
feature_df = feature_df.replace(to_replace="/", value="_per_", regex=True)

In [8]:
# replace nan values with 0
feature_df["Metadata_inducer1_concentration"] = feature_df[
    "Metadata_inducer1_concentration"
].fillna(0)
feature_df["Metadata_inducer2_concentration"] = feature_df[
    "Metadata_inducer2_concentration"
].fillna(0)
feature_df["Metadata_inhibitor_concentration"] = feature_df[
    "Metadata_inhibitor_concentration"
].fillna(0)

#### Combine Inducer1 and Inducer2 into one column

In [9]:
# treatment column merge
conditions = [
    (feature_df["Metadata_inducer2"].isnull()),
    feature_df["Metadata_inducer2"].notnull(),
]

results = [
    (feature_df["Metadata_inducer1"]).astype(str),
    (feature_df["Metadata_inducer1"] + "_" + feature_df["Metadata_inducer2"]).astype(
        str
    ),
]
feature_df["Metadata_Treatment"] = np.select(condlist=conditions, choicelist=results)

# dose column merge
conditions = [
    (feature_df["Metadata_inducer2"].isnull()),
    feature_df["Metadata_inducer2"].notnull(),
]

results = [
    (feature_df["Metadata_inducer1_concentration"].astype(str)).astype(str),
    (
        feature_df["Metadata_inducer1_concentration"].astype(str)
        + "_"
        + feature_df["Metadata_inducer2_concentration"].astype(str)
    ).astype(str),
]
feature_df["Metadata_Dose"] = np.select(condlist=conditions, choicelist=results)

In [10]:
feature_df["Metadata_inducer1_concentration"] = pd.to_numeric(
    feature_df["Metadata_inducer1_concentration"]
)

## N Beta Column condition generation
columns generated to used for linear modeling where terms separated by '__' will be a beta coefficient 

In [11]:
# one beta of inudcer1, inducer1 concentration, inhibitor, and inhibitor concentration all as 1 beta term
feature_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_Dose"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)


# two beta of inducer1, inhibitor, and inhibitor concentration all as 1 beta term + inducer1 concentration as 2nd beta term
feature_df["twob_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
).astype(str)

# three beta of inducer 1 as 1 beta term, inducer1 concentration as 2nd beta term, inhibitor and inhibitor concentration as 3rd beta term
feature_df["threeb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

# four beta of inducer 1 as 1 beta term, inducer1 concentration as 2nd beta term, inhibitor as 3rd beta term, and inhibitor concentration as 4th beta term
feature_df["fourb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

In [12]:
feature_df_table = pa.Table.from_pandas(feature_df)
pq.write_table(feature_df_table, feature_df_out_path)